In [25]:
# bf.get_boot_attrs(connection)

def bytes_to_hex_string(byte_data):
    hex_bytes = [f"{byte:02x}" for byte in bytes(byte_data)]
    return ' '.join(hex_bytes)


In [66]:
from _types import *

# la première commande est : # bf.get_boot_attrs(connection)
# qui fait : 
    # mem_range_response = _send_and_receive(
    #     connection,
    #     Command(CommandCode.GET_MEMORY_ADDRESS_RANGE),
    # )

# Checksum(42).__bytes__()

# len(bytes(Command(CommandCode.GET_MEMORY_ADDRESS_RANGE)))

# # ok, la commande pour demander fait 11 octets, et commence juste par "0b"
# # ensuite, déparser la réponse
v = Version(ResponseCode.SUCCESS)
v.version = 42
v.max_packet_length = 43
v.device_id = 45
v.erase_size = 48
v.write_size = 34

bytes_to_hex_string(v.__bytes__())


'01 00 00 00 00 00 00 00 00 00 00 2a 00 2b 00 00 00 2d 00 00 00 30 00 22 00 00 00 00 00 00 00 00 00 00 00 00 00'

In [ ]:
# Connect to a device in bootloader mode.
connection = serial.Serial(port=<PORT>, baudrate=<BAUDRATE>, timeout=<TIMEOUT>)
# Query its attributes.
bootattrs = bf.get_boot_attrs(connection)
# Load the firmware image and split it into chunks.
total_bytes, chunks = bf.chunked(hexfile=<HEXFILE_PATH_STRING>, bootattrs)
# Erase the device's program memory area.
bf.erase_flash(connection, bootattrs.memory_range, bootattrs.erase_size)

# Write the firmware chunks to the bootloader in a loop.
for chunk in chunks:
    bf.write_flash(connection, chunk)

    # Optionally, check that the write is OK by checksumming.
    if bootattrs.has_checksum:
        bf.checksum(connection, chunk)

    # At this point, you may want to give an indication of the flashing progress,
    # like updating a progress bar.

# Verify that the new application is detected.
bf.self_verify(connection)
